In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import shutil
from sklearn.model_selection import train_test_split

In [3]:
!pip install opendatasets

In [4]:
import opendatasets as od
od.download("https://www.kaggle.com/datasets/sniafas/vyronas-database")

Dataset URL: https://www.kaggle.com/datasets/sniafas/vyronas-database


100%|██████████| 154M/154M [00:05<00:00, 30.3MB/s]


In [5]:
import os
from sklearn.model_selection import train_test_split

real_images_dir = 'vyronas-database/Vyronasdbmin'
sketches_dir = '/content/drive/MyDrive/Pix2Pix_Buildings/sketches_images'

# List of all real images
real_images_list = os.listdir(real_images_dir)

# Create pairs of (real_image, corresponding_sketch)
paired_images = []
for image in real_images_list:
    sketch = image[:-4] + '_sketch.jpg'  # Adjust if there's a different naming convention
    if sketch in os.listdir(sketches_dir):
        paired_images.append((image, sketch))

# Split the pairs into training and validation sets
train_pairs, val_pairs = train_test_split(paired_images, test_size=0.06, random_state=10)

# Separate the pairs into lists of real images and sketches for each set
train_real_images = [pair[0] for pair in train_pairs]
train_sketch_images = [pair[1] for pair in train_pairs]

val_real_images = [pair[0] for pair in val_pairs]
val_sketch_images = [pair[1] for pair in val_pairs]


In [6]:
train_real_images_dir = 'train/real'
train_sketches_dir = 'train/sketches'
val_real_images_dir = 'val/real'
val_sketches_dir = 'val/sketches'

In [7]:
os.makedirs(train_real_images_dir, exist_ok=True)
os.makedirs(train_sketches_dir, exist_ok=True)

os.makedirs(val_real_images_dir, exist_ok=True)
os.makedirs(val_sketches_dir, exist_ok=True)

In [8]:
for image in train_real_images:
    shutil.copy(f'{real_images_dir}/{image}', f'{train_real_images_dir}/{image}')

for image in train_sketch_images:
    shutil.copy(f'{sketches_dir}/{image}', f'{train_sketches_dir}/{image}')

for image in val_real_images:
    shutil.copy(f'{real_images_dir}/{image}', f'{val_real_images_dir}/{image}')

for image in val_sketch_images:
    shutil.copy(f'{sketches_dir}/{image}', f'{val_sketches_dir}/{image}')

In [27]:
!rm -f /content/val/real/*

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import config
from dataset import MapDataset
from utils import save_checkpoint, load_checkpoint, save_some_examples
from generator import Generator
from discriminator import Discriminator
from torch.utils.data import DataLoader
from tqdm import tqdm
from torchvision.utils import save_image

def train_fn(disc, gen, loader, opt_disc, opt_gen, l1, bce, g_scaler, d_scaler):
    loop = tqdm(loader, leave=True)
    for idx, (x, y) in enumerate(loop):
        x, y = x.to(config.DEVICE), y.to(config.DEVICE)

        # Train the Discriminator
        with torch.cuda.amp.autocast():
            y_fake = gen(x)
            D_real = disc(x, y)
            D_fake = disc(x, y_fake.detach())
            D_real_loss = bce(D_real, torch.ones_like(D_real))
            D_fake_loss = bce(D_fake, torch.zeros_like(D_fake))
            D_loss = (D_real_loss + D_fake_loss) / 2  # to make the discriminator train slower relative to the generator

        opt_disc.zero_grad()
        d_scaler.scale(D_loss).backward()
        d_scaler.step(opt_disc)
        d_scaler.update()

        # Train the Generator
        with torch.cuda.amp.autocast():
            D_fake = disc(x, y_fake)
            G_fake_loss = bce(D_fake, torch.ones_like(D_fake))
            L1 = l1(y_fake, y) * config.L1_LAMBDA
            G_loss = G_fake_loss + L1

        opt_gen.zero_grad()
        g_scaler.scale(G_loss).backward()
        g_scaler.step(opt_gen)
        g_scaler.update()

        # Update the tqdm loop description to display the losses
        loop.set_description(f"Epoch [{epoch}/{config.NUM_EPOCHS}]")
        loop.set_postfix(D_loss=D_loss.item(), G_loss=G_loss.item())


In [11]:
disc = Discriminator(in_channels=3).to(config.DEVICE)
gen = Generator(in_channels=3).to(config.DEVICE)
opt_disc = optim.Adam(disc.parameters(), lr=config.LEARNING_RATE, betas=(0.5, 0.999))
opt_gen = optim.Adam(gen.parameters(), lr=config.LEARNING_RATE, betas=(0.5, 0.999))
BCE = nn.BCEWithLogitsLoss()
L1_LOSS = nn.L1Loss()

if config.LOAD_MODEL:
    load_checkpoint(config.CHECKPOINT_GEN, gen, opt_gen, config.LEARNING_RATE)
    load_checkpoint(config.CHECKPOINT_DISC, disc, opt_disc, config.LEARNING_RATE)

train_dataset = MapDataset(real_images_dir='/content/train/real', sketches_dir = '/content/train/sketches')
train_loader = DataLoader(train_dataset, batch_size=config.BATCH_SIZE, shuffle=True, num_workers=config.NUM_WORKERS)
g_scaler = torch.cuda.amp.GradScaler()
d_scaler = torch.cuda.amp.GradScaler()

val_dataset = MapDataset(real_images_dir='/content/val/real', sketches_dir = '/content/val/sketches')
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=True)

for epoch in range(config.NUM_EPOCHS):
    train_fn(disc, gen, train_loader, opt_disc, opt_gen, L1_LOSS, BCE, g_scaler, d_scaler)

    # if epoch % 5 == 0:
    if epoch % 5 == 0:
        save_some_examples(gen, val_loader, epoch, folder='/content/drive/MyDrive/Pix2Pix_Buildings/evaluation')
        save_checkpoint(gen, opt_gen, filename=config.CHECKPOINT_GEN)
        save_checkpoint(disc, opt_disc, filename=config.CHECKPOINT_DISC)

Epoch [0/350]: 100%|██████████| 36/36 [00:15<00:00,  2.38it/s, D_loss=0.582, G_loss=29.2]


=> Saving checkpoint
=> Saving checkpoint


Epoch [5/350]: 100%|██████████| 36/36 [00:15<00:00,  2.36it/s, D_loss=0.705, G_loss=25.8]


=> Saving checkpoint
=> Saving checkpoint


Epoch [10/350]: 100%|██████████| 36/36 [00:14<00:00,  2.46it/s, D_loss=0.609, G_loss=20]


...

Epoch [340/350]: 100%|██████████| 36/36 [00:14<00:00,  2.47it/s, D_loss=0.664, G_loss=5.28]


=> Saving checkpoint
=> Saving checkpoint


Epoch [345/350]: 100%|██████████| 36/36 [00:13<00:00,  2.60it/s, D_loss=0.716, G_loss=5]


=> Saving checkpoint
=> Saving checkpoint


Epoch [349/350]: 100%|██████████| 36/36 [00:13<00:00,  2.67it/s, D_loss=0.601, G_loss=5.47]


In [ ]:
from google.colab import files

# Name of the folder to download
folder_to_download = 'evaluation'

# Zip the folder
!zip -r results.zip {folder_to_download}

# Download the zipped folder
files.download('results.zip')